# ❄️ Snowflake Arctic Quickstart with Cortex LLM Functions

In this quickstart you will learn build and evaluate a RAG application with Snowflake Arctic.

Building and evaluating RAG applications with Snowflake Arctic offers developers a unique opportunity to leverage a top-tier, enterprise-focused LLM that is both cost-effective and open-source. Arctic excels in enterprise tasks like SQL generation and coding, providing a robust foundation for developing intelligent applications with significant cost savings. [Learn more about Snowflake Arctic](https://www.snowflake.com/blog/arctic-open-efficient-foundation-language-models-snowflake/)

In this example, we will use Arctic Embed (`snowflake-arctic-embed-m`) as our embedding model via HuggingFace, and Arctic, a 480B hybrid MoE LLM for both generation and as the LLM to power TruLens feedback functions. The Arctic LLM is fully-mananaged by [Cortex LLM functions](https://docs.snowflake.com/en/user-guide/snowflake-cortex/llm-functions)

Note, you'll need to have an [active Snowflake account](https://signup.snowflake.com/
) to run Cortex LLM functions from Snowflake's data warehouse.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/truera/trulens/blob/main/examples/expositional/models/snowflake_cortex/arctic_quickstart.ipynb)

In [ ]:
# !pip install trulens trulens-providers-cortex chromadb sentence-transformers snowflake-snowpark-python

In [ ]:
import os

from snowflake.snowpark import Session
from trulens.core.utils.keys import check_keys

check_keys("SNOWFLAKE_ACCOUNT", "SNOWFLAKE_USER", "SNOWFLAKE_USER_PASSWORD")


connection_params = {
    "account": os.environ["SNOWFLAKE_ACCOUNT"],
    "user": os.environ["SNOWFLAKE_USER"],
    "password": os.environ["SNOWFLAKE_USER_PASSWORD"],
    "role": os.environ.get("SNOWFLAKE_ROLE", "ENGINEER"),
    "database": os.environ.get("SNOWFLAKE_DATABASE"),
    "schema": os.environ.get("SNOWFLAKE_SCHEMA"),
    "warehouse": os.environ.get("SNOWFLAKE_WAREHOUSE"),
}


# Create a Snowflake session
snowflake_session = Session.builder.configs(connection_params).create()

## Get Data

In this case, we'll just initialize some simple text in the notebook.

In [ ]:
university_info = """
The University of Washington, founded in 1861 in Seattle, is a public research university
with over 45,000 students across three campuses in Seattle, Tacoma, and Bothell.
As the flagship institution of the six public universities in Washington state,
UW encompasses over 500 buildings and 20 million square feet of space,
including one of the largest library systems in the world.
"""

## Create Vector Store

Create a chromadb vector store in memory.

In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("Snowflake/snowflake-arctic-embed-m")

In [ ]:
document_embeddings = model.encode([university_info])

In [ ]:
import chromadb

chroma_client = chromadb.Client()
vector_store = chroma_client.get_or_create_collection(name="Universities")

Add the university_info to the embedding database.

In [ ]:
vector_store.add(
    "uni_info", documents=university_info, embeddings=document_embeddings
)

## Build RAG from scratch

Build a custom RAG from scratch, and add TruLens custom instrumentation.

In [ ]:
from trulens.apps.custom import instrument
from trulens.core import TruSession

session = TruSession()
session.reset_database()

In [ ]:
from snowflake.cortex import Complete


class RAG_from_scratch:
    @instrument
    def retrieve(self, query: str) -> list:
        """
        Retrieve relevant text from vector store.
        """
        results = vector_store.query(
            query_embeddings=model.encode([query], prompt_name="query"),
            n_results=2,
        )
        return results["documents"]

    @instrument
    def generate_completion(self, query: str, context_str: list) -> str:
        """
        Generate answer from context.
        """
        resp = Complete(model='snowflake-arctic', prompt=[{'role': 'user', 'content': 'what is 2 + 2'}], session=snowflake_session)
        return resp

        # def escape_string_for_sql(input_string):
        #     escaped_string = input_string.replace("\\", "\\\\")
        #     escaped_string = escaped_string.replace("'", "''")
        #     return escaped_string

        # prompt = escape_string_for_sql(f"""
        #  We have provided context information below. 
        #     {context_str}
        #     Given this information, please answer the question: {query}
        # """)
        
        # res = snowflake_session.sql(f"""SELECT SNOWFLAKE.CORTEX.COMPLETE(
        #     'snowflake-arctic',
        #     [
        #     {{'role': 'user', 'content': '{prompt}'}}
        #     ], {{
        #         'temperature': 0
        #     }}
        #     )""").collect()    

        # if len(res) == 0:
        #     return "No response from cortex function"
        # completion = json.loads(res[0][0])["choices"][0]["messages"]
        # print("full response from cortex function:")
        # print(res)
        # return completion

    @instrument
    def query(self, query: str) -> str:
        context_str = self.retrieve(query)
        completion = self.generate_completion(query, context_str)
        return completion


rag = RAG_from_scratch()

In [ ]:
# # Define the new version of the function
# import json
# import time
# from typing import Any, Iterator, List, Optional, Union

# from requests import Response
# from snowflake.cortex import _complete
# from snowflake.cortex._sse_client import SSEClient


# def _patched_return_stream_response(response: Response, deadline: Optional[float]) -> dict:
#     client = SSEClient(response)
#     full_content = []  # Accumulate the content here
#     for event in client.events():
#         if deadline is not None and time.time() > deadline:
#             raise TimeoutError()
#         try:
#             message = json.loads(event.data)
#             full_content.append(message["choices"][0]["delta"]["content"])
    
#         except (json.JSONDecodeError, KeyError, IndexError):
#             # For the sake of evolution of the output format,
#             # ignore stream messages that don't match the expected format.
#             pass
#     final_message = {
#                     "id": message["id"],
#                     "created": message["created"],
#                     "model": message["model"],
#                     "tru_content": "".join(full_content),
#                     "usage": message["usage"]
#                 }
#     return final_message


# def _modified_complete_non_streaming_immediate(
#     model: str,
#     prompt,
#     options,
#     session: Optional[Union[str, Session]] = None,
#     deadline: Optional[float] = None,
# ) -> str:
#     response = _complete._complete_rest(model=model, prompt=prompt, options=options, session=session, deadline=deadline)
#     return response

# _complete._return_stream_response = _patched_return_stream_response

# _complete._complete_non_streaming_immediate = _modified_complete_non_streaming_immediate


In [ ]:
# from snowflake.cortex import Complete

# res_str = Complete(model='mistral-7b', prompt=[{'role': 'user', 'content': 'what is 2 + 2'}], session=snowflake_session)
# res_str

## Set up feedback functions.

Here we'll use groundedness, answer relevance and context relevance to detect hallucination.

In [ ]:

import numpy as np
import snowflake.connector
from trulens.core import Feedback
from trulens.core import Select
from trulens.providers.cortex import Cortex

# Create a Snowflake connection
snowflake_connection = snowflake.connector.connect(
    **connection_params
)
provider = Cortex(
    snowflake_connection,
    model_engine="snowflake-arctic",
)


# Define a groundedness feedback function
# f_groundedness = (
#     Feedback(
#         provider.groundedness_measure_with_cot_reasons, name="Groundedness"
#     )
#     .on(Select.RecordCalls.retrieve.rets.collect())
#     .on_output()
# )

# Question/answer relevance between overall question and answer.
# f_answer_relevance = (
#     Feedback(provider.relevance_with_cot_reasons, name="Answer Relevance")
#     .on(Select.RecordCalls.retrieve.args.query)
#     .on_output()
# )

# # Question/statement relevance between question and each context chunk.
# f_context_relevance = (
#     Feedback(
#         provider.context_relevance_with_cot_reasons, name="Context Relevance"
#     )
#     .on(Select.RecordCalls.retrieve.args.query)
#     .on(Select.RecordCalls.retrieve.rets.collect())
#     .aggregate(np.mean)
# )

f_coherence = Feedback(
    provider.coherence_with_cot_reasons, name="coherence"
).on_output()

## Construct the app
Wrap the custom RAG with TruCustomApp, add list of feedbacks for eval

In [ ]:
from trulens.apps.custom import TruCustomApp

tru_rag = TruCustomApp(
    rag,
    app_name="RAG",
    app_version="v0",
    feedbacks=[
        # f_groundedness,
        # f_answer_relevance,
        # f_context_relevance,
        f_coherence,
    ],
)

## Run the app
Use `tru_rag` as a context manager for the custom RAG-from-scratch app.

In [ ]:
with tru_rag as recording:
    resp = rag.query("When is University of Washington founded?")
    

In [ ]:
resp

In [ ]:
session.get_leaderboard(app_ids=[])

In [ ]:
from trulens.dashboard import run_dashboard

run_dashboard(session)